In [13]:
import numpy as np
import mediapipe as mp
import cv2
import mediapipe as mp
import pandas as pd

In [14]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [15]:
filename = 'other_right_jordi'
is_left = False
is_positive = False

SELFIE_INPUT = True
video_capture = cv2.VideoCapture(f'Videos/{filename}.mp4')

landmarks = []
with mp_hands.Hands(static_image_mode=False, 
                    max_num_hands=2,
                    min_detection_confidence=0.7) as hands:
    while video_capture.isOpened():
        success, frame = video_capture.read()
        if not success:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if (SELFIE_INPUT):
            image = cv2.flip(image, 1)
        results = hands.process(image)
        if not results.multi_hand_landmarks:
            continue

        # print(results.multi_handedness)
        # print(len(results.multi_hand_landmarks))
        assert len(results.multi_handedness) == len(results.multi_hand_landmarks)
        for _handedness, hand_landmarks in zip(results.multi_handedness, results.multi_hand_landmarks):
            handedness = _handedness.classification[0]
            landmarks.append((handedness, hand_landmarks.landmark))
        
video_capture.release()

In [16]:
lm_results = [(h.label, h.score, *((lm.x, lm.y, lm.z)
               for lm in lms)) for h, lms in landmarks]
lm_results = [(lms[0], lms[1], *np.ravel(lms[2:])) for lms in lm_results]

In [17]:
landmark_ds = pd.read_csv('Landmark Dataset.csv')

In [18]:
hand_landmark_names = ("WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP",
                       "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", )

dataset = pd.DataFrame(lm_results, columns=('HANDEDNESS',
                                            'SCORE',
                                            *((n+pfx) for n in hand_landmark_names for pfx in ('_X', '_Y', '_Z'))))
dataset['IS_Y'] = is_positive

false_indexes = dataset[dataset.HANDEDNESS !=
                        ('Left' if is_left else 'Right')].index
dataset = dataset.drop(false_indexes)
print(f'Dropped {len(false_indexes)} false rows')


Dropped 11 false rows


In [19]:
landmark_ds = pd.concat((landmark_ds, dataset))
landmark_ds = landmark_ds.drop_duplicates()
landmark_ds.to_csv('Landmark Dataset.csv', index=None)

In [20]:
landmark_ds

,HANDEDNESS,SCORE,WRIST_X,WRIST_Y,WRIST_Z,THUMB_CMC_X,THUMB_CMC_Y,THUMB_CMC_Z,THUMB_MCP_X,THUMB_MCP_Y,...,PINKY_PIP_X,PINKY_PIP_Y,PINKY_PIP_Z,PINKY_DIP_X,PINKY_DIP_Y,PINKY_DIP_Z,PINKY_TIP_X,PINKY_TIP_Y,PINKY_TIP_Z,IS_Y
0,Left,0.998447,0.426867,0.726470,-3.355244e-07,0.475085,0.737292,-0.023625,0.528597,0.722908,...,0.418943,0.521020,-0.062958,0.412330,0.489451,-0.069021,0.409425,0.455898,-0.068255,True
1,Left,0.996158,0.420599,0.726238,-4.918656e-07,0.480033,0.741861,-0.022575,0.530826,0.722220,...,0.407161,0.489169,-0.051421,0.394217,0.445800,-0.055259,0.384253,0.403706,-0.054308,True
2,Left,0.993747,0.415386,0.717941,-5.488089e-07,0.476937,0.729452,-0.019920,0.530574,0.707565,...,0.401667,0.481976,-0.053663,0.387381,0.442011,-0.057296,0.374988,0.402892,-0.056446,True
3,Left,0.998626,0.425512,0.706939,-4.548655e-07,0.471253,0.712519,-0.023218,0.519977,0.688662,...,0.401670,0.494613,-0.055310,0.389583,0.461724,-0.059701,0.380573,0.425952,-0.058264,True
4,Left,0.993242,0.412173,0.703998,-5.492765e-07,0.473183,0.716121,-0.019831,0.526339,0.695925,...,0.398462,0.470240,-0.052493,0.383506,0.429724,-0.056155,0.371041,0.387607,-0.055224,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Right,0.988102,0.684664,0.882696,-5.044337e-07,0.635811,0.862530,-0.014183,0.590339,0.790865,...,0.678128,0.685308,-0.025702,0.677971,0.747536,-0.021155,0.683016,0.788404,-0.011961,False
1916,Right,0.987526,0.683511,0.881183,-5.142140e-07,0.634252,0.861887,-0.013703,0.588673,0.790002,...,0.676364,0.683393,-0.025805,0.676373,0.745343,-0.021178,0.681438,0.786228,-0.012068,False
1917,Right,0.986578,0.682639,0.880318,-5.093490e-07,0.633117,0.860762,-0.014025,0.587662,0.788675,...,0.675310,0.682480,-0.026002,0.674905,0.745055,-0.021436,0.679426,0.786827,-0.012339,False
1918,Right,0.987311,0.681850,0.880446,-5.130564e-07,0.632674,0.861008,-0.013890,0.587123,0.788899,...,0.674119,0.683131,-0.025021,0.674089,0.745035,-0.020474,0.678985,0.785607,-0.011457,False
